### Script to create the datastet used by the app

In [1]:
import pandas as pd
import numpy as np
# %pip install kagglehub
import kagglehub

/opt/anaconda3/envs/jbi010/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Download latest emoji dataset version
# path = kagglehub.dataset_download("subinium/emojiimage-dataset")
# print("Path to dataset files:", path)

In [3]:
countries = pd.read_csv("countriesWorld.csv")

emojis = pd.read_csv("2/full_emoji.csv", usecols=["emoji", "unicode", "name", "Apple"])

# keep only country flags (e.g. no pride flag)
flags = emojis[emojis["name"].str.contains("flag:") == True]
# clean up emoji name
flags["Country"] = flags["name"].str.split(": ").str[1]
flags["CountryCleaned"] = flags["Country"].apply(
    lambda x: x.lower().strip()
)

# remove old emoji name column
flags.drop(columns=["name"], inplace=True)

flags.head(3)

/var/folders/6r/fy40h3sj5sz87ctgpzyw1lgm0000gn/T/ipykernel_21181/972653756.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flags["Country"] = flags["name"].str.split(": ").str[1]
/var/folders/6r/fy40h3sj5sz87ctgpzyw1lgm0000gn/T/ipykernel_21181/972653756.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flags["CountryCleaned"] = flags["Country"].apply(
/opt/anaconda3/envs/jbi010/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

,emoji,unicode,Apple,Country,CountryCleaned
1555,🇦🇨,U+1F1E6 U+1F1E8,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Ascension Island,ascension island
1556,🇦🇩,U+1F1E6 U+1F1E9,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Andorra,andorra
1557,🇦🇪,U+1F1E6 U+1F1EA,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",United Arab Emirates,united arab emirates


In [4]:
countries.head(5)

,Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,"48,0","0,00","23,06","163,07",700.0,"36,0","3,2","12,13","0,22","87,65",1,"46,6","20,34","0,38","0,24","0,38"
1,Albania,EASTERN EUROPE,3581655,28748,"124,6","1,26","-4,93","21,52",4500.0,"86,5","71,2","21,09","4,42","74,49",3,"15,11","5,22","0,232","0,188","0,579"
2,Algeria,NORTHERN AFRICA,32930091,2381740,"13,8","0,04","-0,39",31,6000.0,"70,0","78,1","3,22","0,25","96,53",1,"17,14","4,61","0,101","0,6","0,298"
3,American Samoa,OCEANIA,57794,199,"290,4","58,29","-20,71","9,27",8000.0,"97,0","259,5",10,15,75,2,"22,46","3,27",NaN,NaN,NaN
4,Andorra,WESTERN EUROPE,71201,468,"152,1","0,00","6,6","4,05",19000.0,"100,0","497,2","2,22",0,"97,78",3,"8,71","6,25",NaN,NaN,NaN


In [5]:
# clean up country names
countries["CountryCleaned"] = countries["Country"].apply(
    lambda x: x.lower().strip()
) 

countries["CountryCleaned"]

0         afghanistan
1             albania
2             algeria
3      american samoa
4             andorra
            ...      
222         west bank
223    western sahara
224             yemen
225            zambia
226          zimbabwe
Name: CountryCleaned, Length: 227, dtype: object

In [6]:
# Merge the two DataFrames on the 'CountryCleaned' column
# This will keep only the rows with matching country names
# and drop the rows with mismatched country names
merged = pd.merge(countries, flags, on="CountryCleaned", how="inner") 

# Save the matches to a new DataFrame
matches = merged.copy()

# Identify mismatches in the 'flags' DataFrame
flags_mismatches = flags[~flags["CountryCleaned"].isin(merged["CountryCleaned"])]

# Display the matches and mismatches for review
matches.shape

(196, 25)

In [7]:
flags_mismatches.shape

(65, 5)

In [8]:
flags_mismatches

,emoji,unicode,Apple,Country,CountryCleaned
1555,🇦🇨,U+1F1E6 U+1F1E8,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Ascension Island,ascension island
1564,🇦🇶,U+1F1E6 U+1F1F6,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Antarctica,antarctica
1570,🇦🇽,U+1F1E6 U+1F1FD,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Åland Islands,åland islands
1581,🇧🇱,U+1F1E7 U+1F1F1,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",St. Barthélemy,st. barthélemy
1585,🇧🇶,U+1F1E7 U+1F1F6,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Caribbean Netherlands,caribbean netherlands
...,...,...,...,...,...
1805,🇼🇫,U+1F1FC U+1F1EB,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Wallis & Futuna,wallis & futuna
1807,🇽🇰,U+1F1FD U+1F1F0,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Kosovo,kosovo
1813,🏴󠁧󠁢󠁥󠁮󠁧󠁿,U+1F3F4 U+E0067 U+E0062 U+E0065 U+E006E U+E006...,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",England,england
1814,🏴󠁧󠁢󠁳󠁣󠁴󠁿,U+1F3F4 U+E0067 U+E0062 U+E0073 U+E0063 U+E007...,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Scotland,scotland


In [ ]:
from rapidfuzz import process, fuzz

# Function to find the closest matches for each mismatched flag
def find_closest_matches(mismatches, reference, column_name):
    closest_matches = []
    for country in mismatches[column_name]:
        match = process.extractOne(country, reference[column_name].tolist(), scorer=fuzz.ratio)
        closest_matches.append((country, match[0], match[1]))  # (mismatched, closest match, similarity score)
    return closest_matches

# Find closest matches for flags mismatches
flags_closest_matches = find_closest_matches(flags_mismatches, countries, "CountryCleaned")

# Convert to a DataFrame for easier review
flags_review = pd.DataFrame(flags_closest_matches, columns=["FlagCountry", "ClosestMatch", "SimilarityScore"])

# Sort by similarity score for better review
flags_review = flags_review.sort_values(by="SimilarityScore", ascending=False)

# Display the review DataFrame
print(flags_review.head(10))

# Save to CSV for manual review
flags_review.to_csv("flags_review_.csv", index=False, sep=",", float_format="%.2f", encoding="utf-8")

                 FlagCountry          ClosestMatch  SimilarityScore
39     st. pierre & miquelon  st pierre & miquelon        97.560976
27         st. kitts & nevis   saint kitts & nevis        88.888889
60           wallis & futuna     wallis and futuna        87.500000
50    turks & caicos islands     turks & caicos is        87.179487
9   central african republic  central african rep.        86.363636
42                   réunion               reunion        85.714286
58    british virgin islands    british virgin is.        85.000000
59       u.s. virgin islands        virgin islands        84.848485
11             côte d’ivoire         cote d'ivoire        84.615385
46       são tomé & príncipe   sao tome & principe        84.210526


In [18]:
reviewd_matches = pd.read_csv("flags_review.csv", sep=",")
reviewd_matches

,FlagCountry,ClosestMatch,SimilarityScore,Match
0,st. pierre & miquelon,st pierre & miquelon,97.56,1
1,st. kitts & nevis,saint kitts & nevis,88.89,1
2,wallis & futuna,wallis and futuna,87.50,1
3,turks & caicos islands,turks & caicos is,87.18,1
4,central african republic,central african rep.,86.36,1
...,...,...,...,...
60,palestinian territories,east timor,48.48,0
61,timor-leste,east timor,47.62,0
62,svalbard & jan mayen,san marino,46.67,0
63,kosovo,comoros,46.15,0


In [20]:
# Filter rows where Match == 1
approved_matches = reviewd_matches[reviewd_matches["Match"] == 1]

# Update the flags_mismatches DataFrame with the corrected country names
flags_mismatches["CountryCleaned"] = approved_matches["ClosestMatch"]

# Merge the corrected mismatches with the countries DataFrame
corrected_matches = pd.merge(flags_mismatches, countries, on="CountryCleaned", how="inner")

# Append the corrected matches to the original matches DataFrame
final_matches = pd.concat([matches, corrected_matches], ignore_index=True)

final_matches.to_csv(
    "final_matches.csv",
    index=False,
    sep=",",
    float_format="%.2f",
    encoding="utf-8",
)

/var/folders/6r/fy40h3sj5sz87ctgpzyw1lgm0000gn/T/ipykernel_21181/603280073.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flags_mismatches["CountryCleaned"] = approved_matches["ClosestMatch"]


In [ ]:
final_df = final_matches[['Population',
                          'Area (sq. mi.)',
                          'Pop. Density (per sq. mi.)',
                          'Coastline (coast/area ratio)',
                          'Infant mortality (per 1000 births)',
                          'GDP ($ per capita)',
                          'Literacy (%)',
                          'Phones (per 1000)',
                          'Birthrate',
                          'Deathrate',
                          'emoji',
                          'unicode',
                          'Apple',
                          'Country_y']]
                          

In [30]:
# Replace commas with dots in all columns and convert to floats
for column in final_df.columns:
    if final_df[column].dtype == "object":  # Check if the column is of type object (likely contains strings)
        try:
            # Replace commas with dots and convert to float
            final_df[column] = final_df[column].str.replace(",", ".").astype(float)
        except ValueError:
            # Skip columns that cannot be converted
            pass

# Verify the updated data types
print("Updated Data Types:\n")
print(final_df.dtypes)

Updated Data Types:

Population                              int64
Area (sq. mi.)                          int64
Pop. Density (per sq. mi.)            float64
Coastline (coast/area ratio)          float64
Infant mortality (per 1000 births)    float64
GDP ($ per capita)                    float64
Literacy (%)                          float64
Phones (per 1000)                     float64
Birthrate                             float64
Deathrate                             float64
emoji                                  object
unicode                                object
Apple                                  object
Country_y                              object
dtype: object


### Computing a score for each country

The score is computed in three steps:

1. **Normalize Each Factor**:
   Each numerical factor is scaled to a range of 0–1 using min-max scaling to ensure comparability. This is necessary because factors like population (which can be in millions) have much larger magnitudes than percentages (e.g., literacy rates). The scaling formula is:

   $$
   \text{NormalizedFactor}_{ij} = \frac{x_{ij} - \min(x_j)}{\max(x_j) - \min(x_j)}
   $$

   Where:
   - $x_{ij}$: Value of factor $j$ for country $i$.
   - $\min(x_j)$, $\max(x_j)$: Minimum and maximum values of factor $j$ across all countries.

2. **Compute the Weighted Score**:
   The weighted score for each country is computed as a summation of the normalized factors, weighted by their importance:

   $$
   \text{Score}_i = \sum_{j=1}^{n} w_j \cdot \text{NormalizedFactor}_{ij}
   $$

   Where:
   - $w_j$: Weight assigned to factor $j$.

3. **Normalize the Final Score**:
   The raw scores are normalized to a range of 0–25 for the game:

   $$
   \text{NormalizedScore}_i = \left\lceil 25 \cdot \frac{\text{Score}_i - \min(\text{Score})}{\max(\text{Score}) - \min(\text{Score})} \right\rfloor
   $$

   Where:
   - $\min(\text{Score})$, $\max(\text{Score})$: Minimum and maximum raw scores across all countries.

### Summary:
1.  Each factor is normalized first to ensure comparability.
2.  The weighted sum of the normalized factors is computed to get the raw score.
3.  The raw scores are normalized again to a range of 0–25 for the game.

In [ ]:
# Normalize the columns in final_df to a range of 0–1
for factor in final_df.columns:
    if factor in ["emoji", "unicode", "Apple", "Country_y"]: # only scale numeric columns
        continue
    min_value = final_df[factor].min()
    max_value = final_df[factor].max()
    final_df[factor] = (final_df[factor] - min_value) / (max_value - min_value)

/var/folders/6r/fy40h3sj5sz87ctgpzyw1lgm0000gn/T/ipykernel_21181/2646880500.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df[factor] = (final_df[factor] - min_value) / (max_value - min_value)


In [60]:
# weights for each factor
weights = {
    "Population": -0.25,  
    # High population correlates with well-known countries (e.g., China, India).
    
    "Pop. Density (per sq. mi.)": -0.8,

    "Area (sq. mi.)": -1.5,  
    # Larger area makes a country more visible on maps and globally recognized.
    
    "GDP ($ per capita)": -0.7,  
    # Higher GDP indicates economic importance and global influence.
    
    "Literacy (%)": -0.25,  
    # Higher literacy correlates with development and global visibility.
    
    "Phones (per 1000)": -0.15,  
    # More phones indicate better communication infrastructure and development.
    
    "Coastline (coast/area ratio)": -0.1,  
    # Coastal countries are often more tourist-friendly and globally recognized.
    
    "Deathrate": +0.25,  
    # Higher death rates may indicate poorer or less developed countries, making them less known (not always).
    
    "Birthrate": +0.15,  
    # Extremely high birth rates may correlate with less developed countries, making them less globally prominent.
    
    "Infant mortality (per 1000 births)": +0.35,  
    # High infant mortality rates often indicate poorer countries, reducing global visibility.
}

# Calculate the weighted score for each country
def calculate_score(row):
    score = 0
    for factor, weight in weights.items():
        if factor in row and not pd.isnull(row[factor]):  # Check for valid data
            score += row[factor] * weight
    return score

# Apply the scoring function to each row
final_df["Score"] = final_df.apply(calculate_score, axis=1)

# Normalize scores to lie between 1 and 25
min_score = final_df["Score"].min()
max_score = final_df["Score"].max()
final_df["NormalizedScore"] = 1 + ((final_df["Score"] - min_score) / (max_score - min_score)) * 24

# Round to the nearest integer
final_df["NormalizedScore"] = final_df["NormalizedScore"].round().astype(int)


/var/folders/6r/fy40h3sj5sz87ctgpzyw1lgm0000gn/T/ipykernel_21181/3585912261.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df["Score"] = final_df.apply(calculate_score, axis=1)
/var/folders/6r/fy40h3sj5sz87ctgpzyw1lgm0000gn/T/ipykernel_21181/3585912261.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df["NormalizedScore"] = 1 + ((final_df["Score"] - min_score) / (max_score - min_score)) * 24
/var/folders/6r/fy40h3sj5sz87ctgpzyw1lgm0000gn/T/ipykernel_21181/3585912261.py:50: SettingWithC

In [62]:
final_df_sorted = final_df[[
    "emoji",
    "unicode",
    "Apple",
    "Country_y",
    "NormalizedScore"
]].sort_values(
    by="NormalizedScore",
    ascending=False,
)

final_df_sorted.tail(50)

,emoji,unicode,Apple,Country_y,NormalizedScore
45,🇨🇾,U+1F1E8 U+1F1FE,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Cyprus,15
145,🇵🇹,U+1F1F5 U+1F1F9,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Portugal,15
138,🇵🇼,U+1F1F5 U+1F1FC,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Palau,15
115,🇲🇶,U+1F1F2 U+1F1F6,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Martinique,15
94,🇰🇿,U+1F1F0 U+1F1FF,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Kazakhstan,15
97,🇰🇼,U+1F1F0 U+1F1FC,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Kuwait,15
82,🇮🇳,U+1F1EE U+1F1F3,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",India,15
119,🇲🇽,U+1F1F2 U+1F1FD,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Mexico,15
92,🇯🇪,U+1F1EF U+1F1EA,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Jersey,15
16,🇧🇧,U+1F1E7 U+1F1E7,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Barbados,15


In [44]:
# Export the final sorted DataFrame to a JSON file
final_df_sorted.to_json(
    "emoji_flags_data.json",  # File name
    orient="records",         # Each row is a JSON object
    lines=True,               # Write JSON objects line by line
    force_ascii=False         # Ensure non-ASCII characters (like emojis) are preserved
)